Check with this tutorials:
https://www.youtube.com/watch?v=W_xh6qNSfAQ
https://docs.unsloth.ai/models/tutorials-how-to-fine-tune-and-run-llms/tutorial-how-to-finetune-llama-3-and-use-in-ollama

some models:
4bit pre quantized models we support for 4x faster downloading + no OOMs.
- "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
- "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
- "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
- "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
- "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
- "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
- "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
- "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
- "unsloth/Phi-3-medium-4k-instruct",
- "unsloth/gemma-2-9b-bnb-4bit",
- "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
 More models at https://huggingface.co/unsloth

In [1]:
from unsloth import FastLanguageModel

model_id = 'unsloth/mistral-7b-v0.3'
max_seq_length = 2048 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\temp\finetuningSocratic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1125 07:15:56.267000 26288 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


c:\temp\finetuningSocratic\.venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:348: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE_TORCH}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.11.3: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 5060 Ti. Num GPUs = 1. Max memory: 15.928 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Load the data from the JSONL file

In [2]:
import json
from datasets import Dataset

# Load the training data
training_data = []
with open("Data/socratic_questions_GPTOSS3000.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        training_data.append(json.loads(line))

ds = Dataset.from_list(training_data)
ds[0]



{'claim': '"Arizona officials caught changing ballots, have been arrested."',
 'summary': 'The claim states that Arizona officials were caught changing ballots and have been arrested. It is a qualitative claim about a specific event, with clear geography (Arizona) but no source or methodological details and no explicit time frame, limiting its immediate checkability.',
 'alerts': ['qualitative claim',
  'methodological details absent',
  'time period missing'],
 'url_used': False,
 'question': 'What concrete evidence (e.g., police reports, court documents, reputable news coverage) supports the assertion that officials were caught changing ballots, and how can we verify its authenticity?',
 'history': ['From whose perspective is this claim being presented, and are there other viewpoints—such as statements from law‑enforcement agencies or the officials themselves—that might provide a more balanced understanding?',
  'If the claim were true, what legal and procedural consequences would fo

Use the alpaca prompt

In [3]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Llama/Mistral EOS like <|end_of_text|>

Convert text to Alpaca stule prompt

In [4]:
def to_alpaca_text(examples):
    texts = []

    for claim, summary, alerts, history, question in zip(
        examples["claim"],
        examples["summary"],
        examples["alerts"],
        examples.get("history", [[]]*len(examples["claim"])),
        examples["question"],
    ):
        # 1) instruction: fixed goal of the model
        instruction = (
            "Ask a critical Socratic question that helps analyze everything known so far about the claim. "
            "Differentiate the question from previous questions asked about the claim."
        )

        # 2) input: merge your fields into one context string
        alerts_str = ", ".join(alerts) if isinstance(alerts, list) else str(alerts)
        history_str = "\n- " + "\n- ".join(history) if isinstance(history, list) and len(history) else ""

        input_text = (
            f"[[The claim is {claim}.]]\n"
            f"[[This is all the information known about the claim: {summary}.]]\n"
            f"[[These alerts point out omissions: {alerts_str}.]]\n"
            f"[[Take into account previous questions asked: {history_str}]]"
        )

        # 3) output: your label
        output_text = question

        # final alpaca-formatted training text
        text = alpaca_prompt.format(instruction, input_text, output_text) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

dataset = ds.map(to_alpaca_text, batched=True, remove_columns=ds.column_names)

# check one
print(dataset[0]["text"][:800])

Map: 100%|██████████| 11999/11999 [00:00<00:00, 90645.47 examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Ask a critical Socratic question that helps analyze everything known so far about the claim. Differentiate the question from previous questions asked about the claim.

### Input:
[[The claim is "Arizona officials caught changing ballots, have been arrested.".]]
[[This is all the information known about the claim: The claim states that Arizona officials were caught changing ballots and have been arrested. It is a qualitative claim about a specific event, with clear geography (Arizona) but no source or methodological details and no explicit time frame, limiting its immediate checkability..]]
[[These alerts point out om


In [5]:
from unsloth import FastLanguageModel

# Add LoRA adapters (include embed_tokens + lm_head if using base model)
target_modules = [
    "q_proj","k_proj","v_proj","o_proj",
    "gate_proj","up_proj","down_proj",
] 

model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=target_modules,
    lora_alpha=64*2,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)

Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Train the model

In [8]:
from unsloth import is_bfloat16_supported

# test if bfloat16 is supported
print(is_bfloat16_supported())

True


In [ ]:
from trl import SFTTrainer, SFTConfig
from unsloth import unsloth_train, is_bfloat16_supported

#pretokenize the dataset (faster training)
def tokenize_fn(batch):
    out = tokenizer(
        batch["text"],
        truncation=True,
        max_length=max_seq_length,
        padding=False,
    )
    out["labels"] = out["input_ids"]
    return out
tok_ds = dataset.map(tokenize_fn, batched=True, num_proc=None)

# split the dataset in a train and test set
new_dataset = tok_ds.train_test_split(test_size = 0.01)

# train the model
trainer = SFTTrainer( 
    model = model,
    train_dataset = new_dataset["train"],
    eval_dataset = new_dataset["test"],
    tokenizer = tokenizer,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing=False,
    args= SFTConfig(
        output_dir="outputs",
        seed=3407,

        # Effective batch size = 2 * 4 = 8
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps = 5,

        # Use epochs instead of max_steps
        num_train_epochs=2,

        # more precise tuning than default 2e-4
        learning_rate=1e-4, 
        lr_scheduler_type="linear",

        # bfloat16 is supported
        bf16=True,

        optim="adamw_8bit",
        weight_decay=0.001,

        logging_steps=10,
        save_steps=200,
        #save_total_limit=2,

        # evaluate every 50 steps
        per_device_eval_batch_size = 2,
        eval_accumulation_steps = 4,
        eval_strategy = "steps",
        eval_steps = 25,   
    ),
)

#trainer.train()
unsloth_train(trainer)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/11879 [00:04<?, ? examples/s]


RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.

### Inference

In [8]:
from unsloth import FastLanguageModel

FastLanguageModel.for_inference(model)
messages = [
    {"role": "user", "content": "The claim is 'Arizona officials caught changing ballots, have been arrested.'.\nThis is all the information know about the claim: The claim asserts that Arizona officials were caught changing ballots and have been arrested. It is a qualitative, absolute statement with no cited source or methodological detail. Geography (Arizona) is specified, but the time frame is not, leading to alerts about the claim\'s qualitative nature, missing source/methodology, and missing time period..\n"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_attention_mask=True
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


Save the lora adapters

In [11]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model\\tokenizer_config.json',
 'lora_model\\special_tokens_map.json',
 'lora_model\\chat_template.jinja',
 'lora_model\\tokenizer.json')

### Load model first before Inference

In [1]:
from unsloth import FastLanguageModel

if True: # Load model first before Inference (Set to False if it is already loaded)
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass


messages = [
    {"role": "user", "content": "The claim is 'Arizona officials caught changing ballots, have been arrested.'.\nThis is all the information know about the claim: The claim asserts that Arizona officials were caught changing ballots and have been arrested. It is a qualitative, absolute statement with no cited source or methodological detail. Geography (Arizona) is specified, but the time frame is not, leading to alerts about the claim\'s qualitative nature, missing source/methodology, and missing time period..\n"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\temp\finetuningSocratic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1124 22:12:00.817000 15616 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


c:\temp\finetuningSocratic\.venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:348: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE_TORCH}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 5060 Ti. Num GPUs = 1. Max memory: 15.928 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


What assumptions are being made about the identity and role of "Arizona officials" in this claim?<|end_of_text|>


### Exporting to Ollama

Before you do this, you need to
- install uv
- download windows compatible files from https://github.com/ggml-org/llama.cpp/releases
- copy: these files to llama.cpp
- copy and rename the llama-quantize.exe files so Unsloth can find it (looks for unix style)
copy llama-quantize.exe llama-quantize (from withing the llama.cpp folder)

In [4]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: C:\Users\Gebruiker\.cache\huggingface\hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:00<00:00, 29589.45it/s]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:18<00:00,  4.54s/it]


Unsloth: Merge process complete. Saved to `c:\temp\finetuningSocratic\model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['Meta-Llama-3.1-8B.BF16.gguf']
Unsloth: [2] Converting GGUF bf16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['Meta-Llama-3.1-8B.Q4_K_M.gguf']
Unsloth: No Ollama template mapping found for model 'unsloth/Meta-Llam

In [1]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3) # Wait for a few seconds for Ollama to load!

Finally load the model in ollama

In [12]:
!ollama create Socratic-8B.Q4_K_M -f ./model/Modelfile_q4_k_m

gathering model components ⠙ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components 
copying file sha256:0f9b0553e6e665b9abc04248ba162d5291a5085c7d7e26e8acbe4fd522c73df3 100% 
parsing GGUF ⠋ gathering model components 
copying file sha256:0f9b0553e6e665b9abc04248ba162d5291a5085c7d7e26e8acbe4fd522c73df3 100% 
parsing GGUF 
using ex

In [15]:
!ollama list

NAME                         ID              SIZE      MODIFIED       
Socratic-8B.Q4_K_M:latest    726a00653f0b    4.9 GB    24 seconds ago    
qwen3:4b                     e55aed6fe643    2.5 GB    2 months ago      
